## Write cells and nuclei in .mtx format 

_9 May 2023_

## Importing

### Modules

In [1]:
import numpy as np
import scipy as sp
import scanpy as sc
import pandas as pd
import pickle as pkl
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib import colors
import seaborn as sb
from collections import defaultdict
from numpy import asarray as ar
from collections import Counter

#sklearn <- machine learning
#statsmodels

sc.settings.verbosity = 1
sc.logging.print_version_and_date()
%load_ext autoreload
%autoreload 2

Running Scanpy 1.8.2, on 2023-05-15 00:43.


In [2]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all" # to show output from all the lines in a cells

In [3]:
pd.set_option('display.max_column',None) # display all the columns in pandas

In [4]:
pd.options.display.max_rows = 100

In [5]:
from datetime import date
today = str(date.today())

In [6]:
res_folder = "/nfs/team205/vk8/processed_data/muscle/forDE_LMM/"

In [7]:
import matplotlib
from matplotlib import rcParams
rcParams['pdf.fonttype'] = 42
sc.settings.set_figure_params(dpi = 200, color_map = 'RdPu', dpi_save = 300, vector_friendly = True, format = 'pdf')

## Save adata

In [8]:
adata = sc.read("/nfs/team205/vk8/processed_data/muscle/data_v3/ICM_scell2snuclei_data_2023-05-15.h5ad")

In [10]:
exclude_lev1 = ['CD4+CD8+T-naive', 'Cyc-CD14+Mono', 'Cyc-NK', 'Cyc-T', 'Cyc-cDC2', 'Cyc_B-cell', 'Cyc_Plasma',
          'Hyb', 'mural-CXCL1+', 'Vein-Lymph-CD8A+', 'Mϕ_HLAIIhi', 'Eosinophil', 'Mesothelium', 'pDC', 'MF_type-MYH8+', 'MF_type-RASA4+']

In [11]:
adata_cells = adata[adata.obs['batch']=='cells',:].copy()
adata_nuclei = adata[adata.obs['batch']=='nuclei',:].copy()

In [14]:
adata_cells.obs['annotation_level1'].value_counts()
adata_cells.obs['annotation_level2'].value_counts()
#adata_nuclei.obs['annotation_level1'].cat.categories

FB                18227
MuSC              17554
SMC                7994
MF-IIsc(fg)        7075
MF-Isc(fg)         5205
CD4+T              4343
CD16+NK            3125
CapEC              2982
Macrophage         2981
VenEC              2707
CD8+T              2532
Pericyte           2484
ArtEC              2463
CD14+Mono          2189
B-cell             1279
cDC2               1074
Neutrophil          975
EnFB                714
LymphEC             687
PnFB                547
MF_type-RASA4+      510
Tenocyte            487
mSchwann            486
CD16-NK             466
CD16+Mono           444
Mast                409
B-plasma            310
Mesothelium         203
nmSchwann           150
cDC1                119
MF_type-MYH8+       106
pDC                  39
Eosinophil           36
Name: annotation_level1, dtype: int64

MuSCs                   13551
Inter_FB                 8627
MF_typeII(fg)            6885
Par_FB                   5773
MF_typeI(fg)             4859
Adv_FB                   3827
CD4+T                    3227
CD16+NK                  3075
Mϕ_CD16hi                2561
SMC-CCL2+                2344
Vein-CCL2+               2336
SMC-RAMP1+               2333
SMC                      2266
CD14+Mono                2126
ICAM1+MuSCs              1970
TNFRSF12A+MuSCs          1902
Arteriole                1708
CD8+T-CRTAM+             1572
Pericyte                 1469
Cap-Ven                  1181
Cap                      1162
SMC-PC                   1051
CD4+CD8+T-naive          1028
cDC2                      984
CD8+T                     960
MatNeu                    906
B_naive                   862
Cap-CCL2+                 639
Perineural_FB             547
MF_type-RASA4+            510
Tenocyte                  487
mSchwann_cell             486
CD16-NK                   466
Vein-Lymph

In [18]:
adata_nuclei.obs['annotation_level1'].value_counts()
adata_nuclei.obs['annotation_level2'].value_counts()
#adata_nuclei.obs['annotation_level1'].cat.categories

MF-I             42052
MF-II            28558
FB                9698
MuSC              2687
Macrophage        1742
NMJ_accessory     1309
MTJ                954
MF-Isn(fg)         675
Hyb                414
CD4+T              412
NMJ                406
PnFB               387
CapEC              379
Tenocyte           377
Pericyte           313
CD14+Mono          305
MF-IIsn(fg)        258
Mast               222
Adipocyte          137
CD16+NK            134
VenEC              132
EnFB               128
CD8+T               98
cDC2                84
LymphEC             82
ArtEC               76
SMC                 55
CD16+Mono           52
B-cell              47
cDC1                33
nmSchwann           33
B-plasma            20
Name: annotation_level1, dtype: int64

MF-I                    24380
MF-II                   15844
I-OTU+TNF+              10586
II-OTU+TNF+              9141
I-FAM                    7086
Inter_FB                 6399
II-FAM                   3573
MuSCs                    2687
Par_FB                   2369
Mϕ_CD16hi                1742
NMJ_accessory            1309
MTJ                       954
Adv_FB                    930
MF-Isn(fg)                675
Hyb                       414
NMJ                       406
Perineural_FB             387
Cap                       379
Tenocyte                  377
CD4+T                     370
Pericyte                  313
CD14+Mono                 305
MF-IIsn(fg)               258
Mast_cell                 222
Adipocyte                 137
CD16+NK                   134
Vein                      132
CD8+T                      98
cDC2                       84
Lymphatic                  82
ArtEC                      76
Endoneural_FB-TAC1+        70
Endoneural_FB-CDH19+       58
SMC       

### Save data for the run

In [15]:
def save_data_de_lmm (adata_input, path = 'SKM_data', ctype = 'annotation_level0', age_cov = "Age_group", 
                      rm_ctypes = []):
    adata = adata_input.copy()
    if len(rm_ctypes)>0:
        print(adata.shape)
        adata = adata[~adata.obs[rm_ctypes[0]].isin(rm_ctypes[1]),:].copy() 
    import scipy.io as spio
    sc.pp.filter_genes(adata, min_cells = 10)
    print(adata.shape)
    adata.obs.rename(columns = {age_cov:'Age_bin', ctype:'celltype'}, inplace = True)
    print(adata.obs.columns)
    #/nfs/team205/vk8/processed_data/muscle/forDE_LMM
    spio.mmwrite(f"/nfs/team205/vk8/scripts/de_lmm_natsuhiko/{path}/cpm.mtx", adata.X)
#    with open(f"/nfs/team205/vk8/scripts/de_lmm_natsuhiko/{path}/cpm.mtx", "rb") as f_in, gzip.open("matrix.mtx.gz", "wb") as f_out:
#        f_out.writelines(f_in)
#    os.remove(f"/nfs/team205/vk8/scripts/de_lmm_natsuhiko/{path}/cpm.mtx")
    adata.var.to_csv(f"/nfs/team205/vk8/scripts/de_lmm_natsuhiko/{path}/features.csv")
    adata.obs.to_csv(f"/nfs/team205/vk8/scripts/de_lmm_natsuhiko/{path}/metadata.csv")
    ctypes = adata.obs['celltype'].cat.categories.tolist()
    ctypes_stat_df = pd.DataFrame()
    for i in ctypes:
        adata_ctype = adata[adata.obs['celltype']==i].copy()
        adata_ctype_group1 = adata_ctype[adata_ctype.obs['Age_bin'] == 'young',:]
        adata_ctype_group2 = adata_ctype[adata_ctype.obs['Age_bin'] == 'old',:]
        n_cells_group1 = adata_ctype_group1.shape[0]
        n_cells_group2 = adata_ctype_group2.shape[0]
        n_exprs_cells_group1 = (adata_ctype_group1.X>0).sum(0)
        n_exprs_cells_group2 = (adata_ctype_group2.X>0).sum(0)
        if n_cells_group1 == 0:
            prop_group1 = np.nan
        else:
            prop_group1 = (n_exprs_cells_group1/n_cells_group1).tolist()[0]

        if n_cells_group2 == 0:
            prop_group2 = np.nan
        else:
            prop_group2 = (n_exprs_cells_group2/n_cells_group2).tolist()[0]
        n_genes = adata_ctype.shape[1]
        ctype_1df = pd.DataFrame({'SYMBOL': adata_ctype.var['SYMBOL'].tolist(), \
                  'celltype': np.repeat(i,adata_ctype.shape[1]).tolist(),\
                                 'prop_young': prop_group1,
                                 'prop_old': prop_group2,
                  'n_cells_young': np.repeat(n_cells_group1,n_genes).tolist(), \
                  'n_cells_old': np.repeat(n_cells_group2,n_genes).tolist()},\
                 index = adata_ctype.var['ENSEMBL'])
        ctypes_stat_df = pd.concat([ctypes_stat_df, ctype_1df], axis = 0)

    ctypes_stat_df['ENSEMBL'] = ctypes_stat_df.index
    ctypes_stat_df.reset_index(inplace=True, drop=True)
    ctypes_stat_df.to_csv(f"/nfs/team205/vk8/scripts/de_lmm_natsuhiko/results_all/{path}_prop_stat.csv", index = False)

In [16]:
save_data_de_lmm (adata_cells, path = 'DE_human_scell_coarse_v3_2023May', ctype = 'annotation_level1', age_cov = "Age_group", 
                      rm_ctypes = ['annotation_level2', ['CD4+CD8+T-naive', 'Cyc-CD14+Mono', 'Cyc-NK', 'Cyc-T', 'Cyc-cDC2', 'Cyc_B-cell', 'Cyc_Plasma',
                                                         'Hyb', 'mural-CXCL1+', 'Vein-Lymph-CD8A+', 'Mϕ_HLAIIhi', 'Eosinophil', 'Mesothelium', 'pDC','MF_type-MYH8+', 'MF_type-RASA4+']])

(90902, 33538)
(87725, 23176)
Index(['SampleID', 'barcode', 'concat_sample_no', 'DonorID', 'Sex', 'Age',
       'Species', 'Operator', 'Sample', '10X_version', 'n_counts',
       'n_counts_raw', 'percent_soup', 'n_counts_spliced',
       'n_counts_unspliced', 'percent_spliced', 'n_genes', 'percent_mito',
       'percent_ribo', 'percent_hb', 'Age_bin', 'scrublet_score',
       'scrublet_cluster_score', 'bh_pval', 'is_doublet', 'S_score',
       'G2M_score', 'phase', 'annotation_level0', 'celltype',
       'annotation_level2', 'batch', 'Donor_type', 'BMI', 'Ventilation_cat',
       'BMI_num'],
      dtype='object')


In [17]:
save_data_de_lmm (adata_cells, path = 'DE_human_scell_granular_v3_2023May', ctype = 'annotation_level2', age_cov = "Age_group")

(90902, 23419)
Index(['SampleID', 'barcode', 'concat_sample_no', 'DonorID', 'Sex', 'Age',
       'Species', 'Operator', 'Sample', '10X_version', 'n_counts',
       'n_counts_raw', 'percent_soup', 'n_counts_spliced',
       'n_counts_unspliced', 'percent_spliced', 'n_genes', 'percent_mito',
       'percent_ribo', 'percent_hb', 'Age_bin', 'scrublet_score',
       'scrublet_cluster_score', 'bh_pval', 'is_doublet', 'S_score',
       'G2M_score', 'phase', 'annotation_level0', 'annotation_level1',
       'celltype', 'batch', 'Donor_type', 'BMI', 'Ventilation_cat', 'BMI_num'],
      dtype='object')


In [15]:
save_data_de_lmm (adata_nuclei, path = 'DE_human_snuclei_coarse_v2.1_2023May', ctype = 'annotation_level1', age_cov = "Age_group", 
                      rm_ctypes = ['annotation_level2', ['CD4+CD8+T-naive', 'Cyc-CD14+Mono', 'Cyc-NK', 'Cyc-T', 'Cyc-cDC2', 'Cyc_B-cell', 'Cyc_Plasma',
          'mural-CXCL1+', 'Vein-Lymph-CD8A+', 'Mϕ_HLAIIhi', 'Eosinophil', 'Mesothelium', 'pDC',
          'Lymphatic', 'MF_type-MYH8+', 'MF_type-RASA4+']])

(92259, 33538)
(92135, 25624)
Index(['SampleID', 'barcode', 'concat_sample_no', 'DonorID', 'Sex', 'Age',
       'Species', 'Operator', 'Sample', '10X_version', 'n_counts',
       'n_counts_raw', 'percent_soup', 'n_counts_spliced',
       'n_counts_unspliced', 'percent_spliced', 'n_genes', 'percent_mito',
       'percent_ribo', 'percent_hb', 'Age_bin', 'scrublet_score',
       'scrublet_cluster_score', 'bh_pval', 'is_doublet', 'S_score',
       'G2M_score', 'phase', 'annotation_level0', 'celltype',
       'annotation_level2', 'batch', 'Donor_type', 'BMI', 'Ventilation_cat',
       'BMI_num'],
      dtype='object')


In [16]:
save_data_de_lmm (adata_nuclei, path = 'DE_human_snuclei_granular_v2_2023May', ctype = 'annotation_level2', age_cov = "Age_group")

(92259, 25635)
Index(['SampleID', 'barcode', 'concat_sample_no', 'DonorID', 'Sex', 'Age',
       'Species', 'Operator', 'Sample', '10X_version', 'n_counts',
       'n_counts_raw', 'percent_soup', 'n_counts_spliced',
       'n_counts_unspliced', 'percent_spliced', 'n_genes', 'percent_mito',
       'percent_ribo', 'percent_hb', 'Age_bin', 'scrublet_score',
       'scrublet_cluster_score', 'bh_pval', 'is_doublet', 'S_score',
       'G2M_score', 'phase', 'annotation_level0', 'annotation_level1',
       'celltype', 'batch', 'Donor_type', 'BMI', 'Ventilation_cat', 'BMI_num'],
      dtype='object')


In [15]:
save_data_de_lmm (adata_cells, path = 'DE_human_scell_coarse_v2.1_2023May', ctype = 'annotation_level1', age_cov = "Age_group", 
                      rm_ctypes = ['annotation_level2', ['CD4+CD8+T-naive', 'Cyc-CD14+Mono', 'Cyc-NK', 'Cyc-T', 'Cyc-cDC2', 'Cyc_B-cell', 'Cyc_Plasma',
                                                         'Hyb', 'mural-CXCL1+', 'Vein-Lymph-CD8A+', 'Mϕ_HLAIIhi', 'Eosinophil', 'Mesothelium', 'pDC',
                                                         'Lymphatic', 'MF_type-MYH8+', 'MF_type-RASA4+']])

(90661, 33538)
(87484, 23154)
Index(['SampleID', 'barcode', 'concat_sample_no', 'DonorID', 'Sex', 'Age',
       'Species', 'Operator', 'Sample', '10X_version', 'n_counts',
       'n_counts_raw', 'percent_soup', 'n_counts_spliced',
       'n_counts_unspliced', 'percent_spliced', 'n_genes', 'percent_mito',
       'percent_ribo', 'percent_hb', 'Age_bin', 'scrublet_score',
       'scrublet_cluster_score', 'bh_pval', 'is_doublet', 'S_score',
       'G2M_score', 'phase', 'annotation_level0', 'celltype',
       'annotation_level2', 'batch', 'Donor_type', 'BMI', 'Ventilation_cat',
       'BMI_num'],
      dtype='object')


In [16]:
save_data_de_lmm (adata_nuclei, path = 'DE_human_snuclei_coarse_v2.1_2023May', ctype = 'annotation_level1', age_cov = "Age_group", 
                      rm_ctypes = ['annotation_level2', ['CD4+CD8+T-naive', 'Cyc-CD14+Mono', 'Cyc-NK', 'Cyc-T', 'Cyc-cDC2', 'Cyc_B-cell', 'Cyc_Plasma',
          'mural-CXCL1+', 'Vein-Lymph-CD8A+', 'Mϕ_HLAIIhi', 'Eosinophil', 'Mesothelium', 'pDC',
          'Lymphatic', 'MF_type-MYH8+', 'MF_type-RASA4+']])

(92259, 33538)
(92135, 25624)
Index(['SampleID', 'barcode', 'concat_sample_no', 'DonorID', 'Sex', 'Age',
       'Species', 'Operator', 'Sample', '10X_version', 'n_counts',
       'n_counts_raw', 'percent_soup', 'n_counts_spliced',
       'n_counts_unspliced', 'percent_spliced', 'n_genes', 'percent_mito',
       'percent_ribo', 'percent_hb', 'Age_bin', 'scrublet_score',
       'scrublet_cluster_score', 'bh_pval', 'is_doublet', 'S_score',
       'G2M_score', 'phase', 'annotation_level0', 'celltype',
       'annotation_level2', 'batch', 'Donor_type', 'BMI', 'Ventilation_cat',
       'BMI_num'],
      dtype='object')
